In [1]:
import zipfile
import html
f = zipfile.ZipFile("NFP.zip")
fnames = f.filelist

In [3]:
len(fnames)

1638308

In [9]:
print(f.open(fnames[1_000_000]).read().decode("utf-8"))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<alto xmlns="http://www.loc.gov/standards/alto/ns-v2#" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/standards/alto/ns-v2# http://www.loc.gov/standards/alto/alto-v2.0.xsd">
<Description>
<MeasurementUnit>pixel</MeasurementUnit>
<OCRProcessing ID="IdOcr"><ocrProcessingStep><processingDateTime>2013-01-28</processingDateTime><processingSoftware><softwareCreator>ABBYY</softwareCreator><softwareName>ABBYY FineReader Engine</softwareName><softwareVersion>10</softwareVersion></processingSoftware></ocrProcessingStep></OCRProcessing>
</Description>
<Styles><TextStyle ID="font0" FONTFAMILY="Times New Roman" FONTSIZE="4"/><TextStyle ID="font1" FONTFAMILY="Times New Roman" FONTSIZE="10"/><TextStyle ID="font2" FONTFAMILY="Times New Roman" FONTSIZE="11"/>
</Styles>
<Layout>
<Page ID="Page1" PHYSICAL_IMG_NR="1" HEIGHT="2199" WIDTH="1599">
<LeftMargin HEIGHT

In [16]:
out = pa.table({"ids": ids, "pages": pages})
parquet.write_table(out, f"{i}.parquet", compression = "zstd", compression_level = 5)


In [13]:
i = 0
import pyarrow as pa
from pyarrow import parquet
while True:
    pages = []
    ids = []
    for j in range(5000):
        print(i, end = "\r")
        r = f.open(fnames[i])
        words = []
        for word in r.read().decode("utf-8").split('CONTENT="')[1:]:
            words.append(word.split('"', 1)[0])
        page = html.unescape(" ".join(words))
        pages.append(page)
        ids.append(fnames[i].filename.replace(".xml", ""))
        i += 1
    out = pa.table({"ids": ids, "pages": pages})
    parquet.write_table(out, f"{i}.parquet", compression = "zstd", compression_level = 5)
    print(f"{i}/{len(fnames)}")

5000/1638308
10000/1638308
15000/1638308
20000/1638308
25000/1638308
30000/1638308
35000/1638308
40000/1638308
45000/1638308
50000/1638308
55000/1638308
60000/1638308
65000/1638308
70000/1638308
75000/1638308
80000/1638308
85000/1638308
90000/1638308
95000/1638308
100000/1638308
105000/1638308
110000/1638308
115000/1638308
120000/1638308
125000/1638308
130000/1638308
135000/1638308
140000/1638308
145000/1638308
150000/1638308
155000/1638308
160000/1638308
165000/1638308
170000/1638308
175000/1638308
180000/1638308
185000/1638308
190000/1638308
195000/1638308
200000/1638308
205000/1638308
210000/1638308
215000/1638308
220000/1638308
225000/1638308
230000/1638308
235000/1638308
240000/1638308
245000/1638308
250000/1638308
255000/1638308
260000/1638308
265000/1638308
270000/1638308
275000/1638308
280000/1638308
285000/1638308
290000/1638308
295000/1638308
300000/1638308
305000/1638308
310000/1638308
315000/1638308
320000/1638308
325000/1638308
330000/1638308
335000/1638308
340000/1638308


IndexError: list index out of range

In [28]:
from pyarrow import parquet
r = parquet.read_table("parquet_files/1015000.parquet")
parquet.write_table(r, "parquet_files/1015000.parquet", compression = "zstd", compression_level=5)

In [14]:
!du -ch parquet_files/

6.4G	parquet_files/
6.4G	total


In [15]:
ls -lSh NFP.zip

-rw-r--r--@ 1 benschmidt  staff    63G Apr  6 14:06 NFP.zip


In [29]:
from pathlib import Path
import duckdb
db = duckdb.connect(":memory")
f = db.query("SELECT * FROM parquet_scan('parquet_files/*.parquet') WHERE pages LIKE '%Beethoven%'")    

In [30]:
f = f.arrow()

RuntimeError: IO Error: Could not read all bytes from file "parquet_files/1015000.parquet": wanted=1706306987 read=14239213

In [127]:
%%time

matches = []
from pyarrow import compute as pc
for p in Path("parquet_files").glob("*.parquet"):
    a = parquet.read_table(p)
    matched = pc.extract_regex(a['pages'], '(?P<who>Beethoven|Brahms|Mozart|Schumann)')
    for chunk in matched.chunks:
        if chunk.null_count < len(chunk):
            matches.append(chunk)
    

CPU times: user 55.8 s, sys: 2.82 s, total: 58.7 s
Wall time: 1min


In [128]:
pa.table({'matches': pa.concat_arrays(matches)})

pyarrow.Table
matches: struct<who: string>
  child 0, who: string
----
matches: [  -- is_valid:      [
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      ...
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false
    ]  -- child 0 type: string
    [
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      ...
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      ""
    ]]

In [117]:
matched = pc.extract_regex(a['pages'], '(?P<all>.{0,35}(?P<who>Beethoven|Brahms|Mozart|Schumann|Haydn).{0,35})')
pc.filter(matched, pc.invert(pc.is_null(matched)))

[
  -- is_valid: all not null
  -- child 0 type: string
    [
      "ner hohen Achtunq für dasH.n- KMen Haydn's zu gchcn. — Das. O-Aerreichische",
      ". Ernst. Z. Arie aus Von-Iusn, von Mozart, gesungen von Fräulein Freytag. 4.",
      "Tieck und Hebbel, Musik von Robert Schumann. Sti-dttheater. „Die neue Magdalen",
      "Tieck und Hebbel. Musik von Robert Schumann. HidulsuS, Bischof von Trier. Hr D",
      "phen, und nach dem Hinscheiden von Beethoven und Lenau mußten wir es leider erl",
      "ü!>miesien Composileur«, nähmlich: Mozart und Rossini, der Kasten hat die Fo",
      "lien, als: Die Oratorien von Josef Haydn „Die Schöpfung," „Die 4 Jahres ­ z",
      "proklamiren ; aber ich» mit unserm Mozart zu vergleichen, wie es seinen.Land",
      " würd- er wieder Ehrenmitglied des Mozarteums in Salzburg. (Ehre dem Ehre ge",
      "die mit demselben unter den Namen „Mozarteum" verbundene Musiklebr-Anstalt i",
      ...
      "zur Darstellung vorbereitet. — Der Mozarteums Direktoi D

In [106]:
matched.

[
  -- is_valid:
      [
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      ...
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false
    ]
  -- child 0 type: string
    [
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      ...
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      "",
      ""
    ]
]